In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report


# Load your CSV data into a pandas DataFrame
df = pd.read_csv('../../../data/processed/tokenized_data.csv')

# Load the vectorized features from the .npy file
vectorized_features = np.load('../../../data/processed/vectorized_features_with_summary_tokens.npy')

feature_vectors = pd.DataFrame(vectorized_features)
feature_vectors.columns = [f'vec_feature_{i}' for i in range(feature_vectors.shape[1])]

X = feature_vectors # Feature vector
y = df['overall']  # Target variable

# Convert class labels to start from 0
y = y - 1


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=5, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.58      0.62      0.60     20917
         1.0       0.49      0.18      0.26     12765
         2.0       0.47      0.36      0.41     19019
         3.0       0.58      0.37      0.45     28682
         4.0       0.74      0.92      0.82     88965

    accuracy                           0.67    170348
   macro avg       0.57      0.49      0.51    170348
weighted avg       0.65      0.67      0.64    170348

